# **1. Overwiew**

# **2. Business and Data Understanding**
## **2.1 Business Problem**



## **2.2 Data Understanding**

### **2.2.1 Dataset Features**
The dataset employed in the study was downloaded from https://data.world/crowdflower/brands-and-product-emotions/file/judge-1377884607_tweet_product_company.csv 

### **2.2.2 Dataset Features**
The following feature about ____ are provided:





## **2.3 Methodology**
The adopted structure for the project was CRISP-DM that entails undertaking Business understanding; Data Understanding; Data Preparation; Data Cleaning and Exploratory Data Analysis (EDA); Modelling; Conclusion and Recommendations.

# **3. Data Cleaning and EDA**
## **3.1 Data Cleaning & Preparation**
Importing packages

In [14]:
# Importing standard packages
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
%matplotlib inline 
from textwrap import fill 

from sklearn.model_selection import train_test_split, cross_val_score, RepeatedStratifiedKFold, GridSearchCV
import warnings
import math
import os
warnings.filterwarnings("ignore")

# from sklearn.pipeline import Pipeline

#classification models
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm

# classification metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import roc_curve, roc_auc_score, auc
from sklearn.metrics import classification_report, ConfusionMatrixDisplay

# scalers
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelBinarizer, label_binarize

# dummies
from sklearn.preprocessing import OneHotEncoder

# NLP libraries
from nltk.corpus import stopwords, wordnet
import string
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer, word_tokenize
import re
from nltk import FreqDist
from nltk import pos_tag
from wordcloud import WordCloud 

# imbalanced-learn
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

In [2]:
# loading the dataset
sentiment_data = pd.read_csv("../data/judge-1377884607_tweet_product_company.csv", encoding="ISO-8859-1")
sentiment_data
#C:\Users\mauka\OneDrive\Desktop\PHASE4\p4_project\data\judge-1377884607_tweet_product_company.csv

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion
...,...,...,...
9088,Ipad everywhere. #SXSW {link},iPad,Positive emotion
9089,"Wave, buzz... RT @mention We interrupt your re...",NaN,No emotion toward brand or product
9090,"Google's Zeiger, a physician never reported po...",NaN,No emotion toward brand or product
9091,Some Verizon iPhone customers complained their...,NaN,No emotion toward brand or product


In [3]:
# checking the data types
sentiment_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9093 entries, 0 to 9092
Data columns (total 3 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   tweet_text                                          9092 non-null   object
 1   emotion_in_tweet_is_directed_at                     3291 non-null   object
 2   is_there_an_emotion_directed_at_a_brand_or_product  9093 non-null   object
dtypes: object(3)
memory usage: 213.2+ KB


In [4]:
# rename the column names for easier analysis
sentiment_data.rename(columns=
                      {
    "tweet_text":"text",
    "emotion_in_tweet_is_directed_at":"product",
    "is_there_an_emotion_directed_at_a_brand_or_product":"sentiment"
},
                      inplace=True)
sentiment_data

,text,product,sentiment
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion
...,...,...,...
9088,Ipad everywhere. #SXSW {link},iPad,Positive emotion
9089,"Wave, buzz... RT @mention We interrupt your re...",NaN,No emotion toward brand or product
9090,"Google's Zeiger, a physician never reported po...",NaN,No emotion toward brand or product
9091,Some Verizon iPhone customers complained their...,NaN,No emotion toward brand or product


In [5]:
#  encode the sentiments column (from test to numerical using label mapping)

# clean whitespaces in the sentiment column
sentiment_data["sentiment"] = sentiment_data["sentiment"].str.strip()

# apply the label mapping
label_mapping = {
    "Negative emotion" : '1',
    "Positive emotion" : '3',
    "No emotion toward brand or product" : '5',
    "I can't tell" : 'irrelevant'  
}

sentiment_data["sentiment"] = sentiment_data["sentiment"].map(label_mapping)

#previewing the 1st 5 rows
sentiment_data.head()

,text,product,sentiment
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,1
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,3
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,3
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,1
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,3


In [6]:
# drop the "product" column as it has too many null values and only use two columns in the analysis. 

# since we are building a model to rate the sentiment if a tweet based on its content, then the feature variable (y) is the "text" column while the target variable (x) is the "sentiment" column

# column to be dropped
drop_col = ["product"]

# dropping the column
sentiment_data = sentiment_data.drop(drop_col, axis=1)

# checking the information of the new dataset
sentiment_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9093 entries, 0 to 9092
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       9092 non-null   object
 1   sentiment  9093 non-null   object
dtypes: object(2)
memory usage: 142.2+ KB


The new dataframe has 2 columns and 9093 rows. Column "text" has a missing value. The dataframe is made up of one datatype: object.

In [7]:
# dropping the null value
sentiment_data = sentiment_data.dropna()

# confirming no null values are remaining
sentiment_data.isna().sum()

text         0
sentiment    0
dtype: int64

In [8]:
# checking the summary statistics of the data
sentiment_data.describe()

,text,sentiment
count,9092,9092
unique,9065,4
top,RT @mention Marissa Mayer: Google Will Connect...,5
freq,5,5388


sentiment has 4 unique categories with 5(no emotion toward brand or product i.e. neutral) taking about 59.26% (5388 out of 9092 entries)  of the data.This creates an imbalance in the dataset.  

sentiment is expected to have 3 labels (1 - Negative emotion, 3 - Positive Emotion, 5 - no emotion) and not 4. There is need to drop the 4th label (irrelevant).

In [9]:
# check for duplicates
sentiment_data[sentiment_data.duplicated()]

,text,sentiment
468,"Before It Even Begins, Apple Wins #SXSW {link}",3
776,Google to Launch Major New Social Network Call...,5
2232,Marissa Mayer: Google Will Connect the Digital...,5
2559,Counting down the days to #sxsw plus strong Ca...,3
3950,Really enjoying the changes in Gowalla 3.0 for...,3
3962,"#SXSW is just starting, #CTIA is around the co...",3
4897,"Oh. My. God. The #SXSW app for iPad is pure, u...",3
5338,RT @mention ÷¼ GO BEYOND BORDERS! ÷_ {link} ...,5
5341,"RT @mention ÷¼ Happy Woman's Day! Make love, ...",5
5881,RT @mention Google to Launch Major New Social ...,5


In [10]:
# drop duplicates
sentiment_data.drop_duplicates(inplace=True)

## **3.2 Data Preprocessing**
To remove stop words, punctuation marks and carry out lemmatization

In [11]:
# define a function to preprocess the data

def get_wordnet_pos(word): #helper function for Part of Speech (POS)  aware lemmatization in nltk 
    """Map POS tag to the first character that lemmatizer() accepts"""
    tag = pos_tag([word])[0][1][0].upper() # gives the POS tag for the word and extracts the first letter of the tag
    tag_dict = {
        "J": wordnet.ADJ,"N": wordnet.NOUN,
        "V": wordnet.VERB, "R": wordnet.ADV
    }
    return tag_dict.get(tag, wordnet.NOUN) # maps the letters to WordNet's lemmitazation frmat .If the POS tag isn't found, it returns (defaults) to noun

def cleaned_text (text):
    stopword_list = stopwords.words("english")
    stopword_list += list(string.punctuation)
    lemmatizer = WordNetLemmatizer()
    
    # regex cleaning - to remove usernames, hyperlinks, hashtags, mentions etc
    text = re.sub(r"https?://\S+|www\.\S+", "", text) #removes links
    text = re.sub(r"@\w+", "", text) # removes mentions
    text = re.sub(r"#", "", text) # remove hashtag (#), but keeps the word
    text = re.sub(r"\.\.+",".", text)  # replaces multiple dots
    
    # tokenize the text
    text = word_tokenize(text)
    
    # lowercase and remove the stopwords
    text = [word.lower() for word in text if word.lower() not in stopword_list] 
    
    #lemmatize with POS tagging
    text = [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in text]
    
    # join preprocessed text as a single string
    text = ' '.join(text)
    
    return text

In [12]:
#call the function and apply to the text column
sentiment_data["cleaned_text"] = sentiment_data["text"].apply(cleaned_text)

sentiment_data[["text", "cleaned_text"]].head()

,text,cleaned_text
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,3g iphone 3 hr tweet rise_austin dead need upg...
1,@jessedee Know about @fludapp ? Awesome iPad/i...,know awesome ipad/iphone app 'll likely apprec...
2,@swonderlin Can not wait for #iPad 2 also. The...,wait ipad 2 also sale sxsw
3,@sxsw I hope this year's festival isn't as cra...,hope year 's festival n't crashy year 's iphon...
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,great stuff fri sxsw marissa mayer google tim ...


In [13]:
sentiment_data

,text,sentiment,cleaned_text
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,1,3g iphone 3 hr tweet rise_austin dead need upg...
1,@jessedee Know about @fludapp ? Awesome iPad/i...,3,know awesome ipad/iphone app 'll likely apprec...
2,@swonderlin Can not wait for #iPad 2 also. The...,3,wait ipad 2 also sale sxsw
3,@sxsw I hope this year's festival isn't as cra...,1,hope year 's festival n't crashy year 's iphon...
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,3,great stuff fri sxsw marissa mayer google tim ...
...,...,...,...
9088,Ipad everywhere. #SXSW {link},3,ipad everywhere sxsw link
9089,"Wave, buzz... RT @mention We interrupt your re...",5,wave buzz rt interrupt regularly schedule sxsw...
9090,"Google's Zeiger, a physician never reported po...",5,google 's zeiger physician never report potent...
9091,Some Verizon iPhone customers complained their...,5,verizon iphone customer complain time fell bac...


In [16]:
save_path = "../data/cleaned_sentiment_data.csv"
sentiment_data.to_csv(save_path, index=False, encoding="ISO-8859-1")